In [1]:
import numpy as np
import pandas as pd
import torch
from WideAndDeep_pytorch.wide_deep import WideAndDeepModel
from WideAndDeep_pytorch.avazu import AvazuDataset
from WideAndDeep_pytorch.train import train,test,EarlyStopper
from torch.utils.data import DataLoader
from IPython.core.interactiveshell import  InteractiveShell
InteractiveShell.ast_node_interactivity='all'
pd.set_option('max_columns',600)
pd.set_option('max_rows',500)
torch.manual_seed(0)

# load dataset
dataset=AvazuDataset('./data/train_150m.csv',rebuild_cache=False)
model=WideAndDeepModel(dataset.field_dims, embed_dim=16,mlp_dims=(16, 16), dropout=0.2)


#split dataset into train/test
train_length = int(len(dataset) * 0.9)
valid_length = int(len(dataset) * 0.1)
print("train_length,valid_length",train_length,valid_length)
test_length = len(dataset) - train_length - valid_length
train_dataset, valid_dataset, test_dataset = torch.utils.data.random_split(
    dataset, (train_length, valid_length, test_length))

#DataLoader
train_data_loader = DataLoader(train_dataset, batch_size=256, num_workers=0)
valid_data_loader = DataLoader(valid_dataset, batch_size=256, num_workers=0)
test_data_loader = DataLoader(test_dataset, batch_size=256, num_workers=0)



criterion = torch.nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001, weight_decay=0.000001)

# Training
train_auc = []
test_auc = []

for epoch_i in range(10):
    device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
    #device = None
    train(model, optimizer, train_data_loader, criterion, device=device)
    auc_train = test(model, train_data_loader, device=device) # test
    auc_valid = test(model, valid_data_loader, device=device)
    #auc_test = test(model, test_data_loader, device=None)
    print('epoch:{}：'.format(epoch_i))
    print('Train AUC:{}'.format(auc_train))
    print('Test AUC:{}'.format(auc_valid))
    train_auc.append(auc_train)
    test_auc.append(auc_valid)

print('train_auc',train_auc)
print('test_auc',test_auc)

Model Architecture
Sequential(
  (0): Linear(in_features=352, out_features=16, bias=True)
  (1): ReLU()
  (2): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): Dropout(p=0.2, inplace=False)
  (4): Linear(in_features=16, out_features=16, bias=True)
  (5): ReLU()
  (6): BatchNorm1d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): Dropout(p=0.2, inplace=False)
  (8): Linear(in_features=16, out_features=1, bias=True)
)
train_length,valid_length 900000 100000


100%|████████████████████████████████████████| 391/391 [00:01<00:00, 370.24it/s]


epoch:0：
Train AUC:0.8105597391608654
Test AUC:0.7629090273954882


100%|████████████████████████████████████████| 391/391 [00:01<00:00, 373.87it/s]


epoch:1：
Train AUC:0.8265656763521745
Test AUC:0.7659153941649471


...

epoch:7：
Train AUC:0.7901475597138979
Test AUC:0.7372659341772564


100%|████████████████████████████████████████| 391/391 [00:01<00:00, 288.45it/s]


epoch:8：
Train AUC:0.7838541090138819
Test AUC:0.7317181154344692


100%|████████████████████████████████████████| 391/391 [00:01<00:00, 359.99it/s]

epoch:9：
Train AUC:0.7844823313160657
Test AUC:0.7339091035210827
train_auc [0.8105597391608654, 0.8265656763521745, 0.8177402340059204, 0.8077062039166588, 0.7959927671296531, 0.795249257597784, 0.791811901289297, 0.7901475597138979, 0.7838541090138819, 0.7844823313160657]
test_auc [0.7629090273954882, 0.7659153941649471, 0.7562652667963806, 0.7490523910157451, 0.7410981605108372, 0.7399745178203014, 0.7384085020627699, 0.7372659341772564, 0.7317181154344692, 0.7339091035210827]
